In [95]:
# Data handling and processing
import pandas as pd
import numpy as np
import math

# Sklearn imports
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# File mgmt
import os

os.chdir('Data')
!ls

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'Data'

In [96]:
path = 'Transactions_Village114/ds3170_tx_All_Data_5208_2019_1211_082948.txt'
df = pd.read_csv(path, sep="\t")
cols = ['Anon Student Id', 'Session Id', 'Duration (sec)', 'Level (Tutor Name)', 'Level (Tutor)', 'Problem Name', 'Problem View','Step Name', 'Attempt At Step','Is Last Attempt','Outcome', 'Input','CF (File)','CF (Matrix)','CF (Matrix Level)', 'CF (Matrix Order)', 'CF (Total Activity Problems)']
df = df[cols]
bubble_pop_df = df[df['Level (Tutor Name)'] == 'bubble_pop']
bpop_math_df = bubble_pop_df[bubble_pop_df['CF (Matrix)'] == 'math'] #6172

# X and y -> inputs and outputs for the classification model
cols = ['Duration (sec)', 'Level (Tutor)', 'Attempt At Step', 'CF (Matrix Level)', 'CF (Matrix Order)', 'Outcome']

C:\Users\jithe\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,7,37,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Helpful functions

In [97]:
# Clean duration (sec) columns: Convert all values to float, if not able to convert set value in that row as duration_mean
def clean_duration_col_with_mean(X):
    strings = 0
    ints = 0
    floats = 0
    other = 0
    cnt = 0
    sums = 0
    posn = []
    X_duration_arr = X['Duration (sec)'].array

    for i in range(len(X_duration_arr)):
        if isinstance(X_duration_arr[i], int):
            ints += 1
        elif isinstance(X_duration_arr[i], str):
            strings += 1
            try:
                X_duration_arr[i] = float(X_duration_arr[i])
            except ValueError:
#                 print("Cant convert to a float", X_duration_arr[i], "Position: ",i)
                posn.append(i)
        elif isinstance(X_duration_arr[i], float):
            floats += 1
    
        if isinstance(X_duration_arr[i], float):
            sums += X_duration_arr[i]
            cnt += 1

    duration_mean = sums/cnt
    for i in range(len(posn)):
        X_duration_arr[posn[i]] = duration_mean

    for i in range(len(X_duration_arr)):
        col = bpop_math_df.columns.get_loc('Duration (sec)')
        val = X_duration_arr[i]
        X.iloc[i, col] = val
    
    return X

# Ordinal encodes a column
def ordinal_encode_col(X, colname):
    enc = OrdinalEncoder()
    level_tutor = X[colname].tolist()
    tutor_level_2dlist = []
    for i in range(len(level_tutor)):
        tutor_level_2dlist.append([level_tutor[i]])

    enc.fit(tutor_level_2dlist)

    col = X.columns.get_loc(colname)
    num_entries = X.count()[colname]
    for i in range(num_entries):
        val = X.iloc[i, col]
        val = enc.transform([[val]])[0][0]
        X.iloc[i, col] = val
    
    return X

def rfe(X_train, y_train):
    data_final_vars=X_train.columns.values.tolist()
    y_=['y']
    X=[i for i in data_final_vars if i not in y_]
    logreg = LogisticRegression()
    rfe = RFE(logreg, 20)
    rfe = rfe.fit(X_train, y_train)
#     print("RFE Support: ",rfe.support_)
#     print("RFE Ranking: ", rfe.ranking_)

# incorrect -> 0, correct -> 1
# set_zero = "INCORRECT" means incorrect will be set to 0 and other value as 1
def encode_outputs(y, colname, set_zero):
    col = y.columns.get_loc(colname)
    for i in range(y.count()[colname]): 
        val = y.iloc[i, col]
        if val == set_zero:
            val = 0
        else: 
            val = 1
        y.iloc[i, col] = val
    
    return y

def run_logistic_regression(X, y, clean_method, encoder, regularization, test_split, apply_rfe):
    
    if clean_method == "mean":
        X = clean_duration_col_with_mean(X)
    
    if encoder == "ordinal":
        X = ordinal_encode_col(X, 'Level (Tutor)')
    
    y = encode_outputs(y, 'Outcome', 'INCORRECT').values.ravel()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split)

    if apply_rfe:
        rfe(X_train, y_train)
        
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    performance = 100 * logreg.score(X_test, y_test)
#     print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(performance))
    return performance
    

In [98]:
clean_method = "mean" # mean median or mode
encoder = "ordinal" # ordinal, onehot etc..
regularization = "none" # none, l1, l2 etc..
test_split = 0.2
apply_rfe = True
num_runs = 10

total_perfs = []

for i in range(num_runs):
    X = bpop_math_df[cols]
    X = X.sample(frac=1).reset_index(drop=True)
    y = X[['Outcome']]
    X = X.drop(['Outcome'], axis=1)
    perf = run_logistic_regression(X, y, clean_method, encoder, regularization, test_split, apply_rfe)
    total_perfs.append(perf)
    print(total_perfs)

avg_perf = sum(total_perfs)/num_runs
print("Avg. performance over ", num_runs, "runs: ", avg_perf)

[70.76923076923077]
[70.76923076923077, 69.63562753036437]
[70.76923076923077, 69.63562753036437, 69.23076923076923]
[70.76923076923077, 69.63562753036437, 69.23076923076923, 70.8502024291498]
[70.76923076923077, 69.63562753036437, 69.23076923076923, 70.8502024291498, 70.68825910931174]
[70.76923076923077, 69.63562753036437, 69.23076923076923, 70.8502024291498, 70.68825910931174, 67.61133603238866]
[70.76923076923077, 69.63562753036437, 69.23076923076923, 70.8502024291498, 70.68825910931174, 67.61133603238866, 69.7165991902834]
[70.76923076923077, 69.63562753036437, 69.23076923076923, 70.8502024291498, 70.68825910931174, 67.61133603238866, 69.7165991902834, 68.17813765182186]
[70.76923076923077, 69.63562753036437, 69.23076923076923, 70.8502024291498, 70.68825910931174, 67.61133603238866, 69.7165991902834, 68.17813765182186, 68.50202429149797]
[70.76923076923077, 69.63562753036437, 69.23076923076923, 70.8502024291498, 70.68825910931174, 67.61133603238866, 69.7165991902834, 68.1781376518

In [100]:
print(max(total_perfs), min(total_perfs))

70.93117408906883 67.61133603238866
